In [1]:
import os
import zipfile

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import pandas as pd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import xarray as xr

In [2]:
# ----- Parameters -----
# regions = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
regions = [15]

dst_crs = 'EPSG:4326'
deg_res = 0.1

# GCMs and RCP scenarios
gcm_names = ['BCC-CSM2-MR', 'CESM2', 'CESM2-WACCM', 'EC-Earth3', 'EC-Earth3-Veg', 'FGOALS-f3-L', 
             'GFDL-ESM4', 'INM-CM4-8', 'INM-CM5-0', 'MPI-ESM1-2-HR', 'MRI-ESM2-0', 'NorESM2-MM']
rcps = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

binned_fp_zipped = '/Users/drounce/Documents/HiMAT/spc_backup/simulations-binned/_zipped/'
binned_fp_prefix = '/Users/drounce/Documents/HiMAT/spc_backup/simulations-binned/_unzipped/'
oggm_gdirs_fp_prefix = '/Users/drounce/Documents/HiMAT/oggm_gdirs/per_glacier-all/'

latlon_csv_fp = '/Users/drounce/Documents/HiMAT/spc_backup/latlon_unique_csvs/'

In [3]:
def reproject_raster2wgs84(in_fullfn, dst_fullfn, dst_crs):
    """
    in_fullfn : input full filename of raster
    dst_fullfn: destination full filename to write raster
    dst_crs   : destination coordinate reference system (e.g,. 'EPSG:4326')
    """
    
    with rasterio.open(in_fullfn) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(dst_fullfn, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

def show_fig(image, title, color, ctitle, bounds=None, res=None, vmin=None, vmax=None, savefig=False):
    fig, ax = plt.subplots(figsize=(12,6))
    c = ax.imshow(image, cmap=color, extent=bounds, vmin=vmin, vmax=vmax)
    fig.colorbar(c, label=ctitle)
    fig.suptitle(title)
    plt.show()
    if savefig == True:
        fig.savefig(title + '.jpg', dpi=1000) # to save the plot as a jpg image  

<font face="Calibri" size="3"> <b>Process each glacier's binned data into fraction of area in each lat/lon grid cell</b> </font>

In [4]:
# Process data
for reg in regions:
    for rcp in rcps:
        for gcm_name in gcm_names:
            
            binned_fp_zipped_reg = binned_fp_zipped + str(reg).zfill(2) + '/binned/'
            
            unzipped_fp = binned_fp_prefix + str(reg).zfill(2) + '/' + gcm_name + '/' + rcp + '/binned/'
            if not os.path.exists(unzipped_fp):
                os.makedirs(unzipped_fp)
            
            if os.path.exists(unzipped_fp):
                # Only unzip data once
                if len(os.listdir(unzipped_fp)) <= 1:
                    zipped_fn = binned_fp_zipped_reg + gcm_name + '_' + rcp + '_binned.zip'
                    with zipfile.ZipFile(zipped_fn, 'r') as zip_ref:
                        zip_ref.extractall(unzipped_fp)
            netcdf_fp = unzipped_fp
            
            netcdf_fns = []
            for i in os.listdir(netcdf_fp):
                if i.endswith('binned.nc'):
                    netcdf_fns.append(i)
            netcdf_fns = sorted(netcdf_fns)
            
            for nfn, netcdf_fn in enumerate(netcdf_fns):
                
                debug=False
                for batman in [0]:
#                 if netcdf_fn.startswith('16.01365'): # not working because not all pixels counted

                    if nfn%1000 == 0:
                        print(netcdf_fn)
            
                    try:
                        ds = xr.open_dataset(netcdf_fp + netcdf_fn)

                        # Reproject DEM (for binning) and Ice Thickness (for glacier mask) into WGS84
                        rgiid = ds.RGIId.values[0]

                        latlon_csv_fp_rcpgcm = latlon_csv_fp + str(reg).zfill(2) + '/' + gcm_name + '/' + rcp + '/'
                        if not os.path.exists(latlon_csv_fp_rcpgcm):
                            os.makedirs(latlon_csv_fp_rcpgcm)
                        latlon_vol_annual_fn = rgiid + '_latlon_vol_annual_' + gcm_name + '_' + rcp + '.csv'

                        if not os.path.exists(latlon_csv_fp_rcpgcm + latlon_vol_annual_fn):

                            oggm_gdirs_fp = oggm_gdirs_fp_prefix + rgiid[0:8] + '/' + rgiid[0:11] + '/' + rgiid + '/'
                            dem_fn = 'dem.tif'
                            thick_fn = 'consensus_h.tif'

                            if os.path.exists(oggm_gdirs_fp + thick_fn):
                                # Reproject thickness
                                reproject_raster2wgs84(oggm_gdirs_fp + thick_fn,
                                                       oggm_gdirs_fp + thick_fn.replace('.tif', '-reprojected.tif'), 
                                                       dst_crs)
                                # Reproject DEM
                                reproject_raster2wgs84(oggm_gdirs_fp + dem_fn,
                                                       oggm_gdirs_fp + dem_fn.replace('.tif', '-reprojected.tif'), 
                                                       dst_crs)

                                # Load reprojected thickness and DEM
                                thick_rs = rasterio.open(oggm_gdirs_fp + thick_fn.replace('.tif', '-reprojected.tif'))
                                dem = rasterio.open(oggm_gdirs_fp + dem_fn.replace('.tif', '-reprojected.tif')).read(1)

                                # Create glacier mask
                                glac_mask = thick_rs.read(1).copy()
                                glac_mask[glac_mask > 0] = 1


                                # ===== COMPUTE UNIQUE LAT/LON =====
                                rast_bounds = thick_rs.bounds
                                bnd_west = rast_bounds[0]
                                bnd_east = rast_bounds[2]
                                bnd_south = rast_bounds[1]
                                bnd_north = rast_bounds[3]

                                # ----- Loop over lat/lons -----
                                # Longitude values
                                deg_res_pix_lon = (bnd_east - bnd_west) / glac_mask.shape[1]
                                lon_values = bnd_west + deg_res_pix_lon/2 + np.arange(0,glac_mask.shape[1]) * deg_res_pix_lon

                                # Latitude values (note: need to reverse them to be consistent with position in array)
                                deg_res_pix_lat = (bnd_north - bnd_south) / glac_mask.shape[0]
                                lat_values = bnd_south + deg_res_pix_lat/2 + np.arange(0,glac_mask.shape[0]) * deg_res_pix_lat
                                lat_values = lat_values[::-1]

                                # Record bounds
                                lon_bndlow = np.floor(bnd_west/deg_res)*deg_res
                                lon_bndhigh = np.ceil(bnd_east/deg_res)*deg_res
                                lat_bndlow = np.floor(bnd_south/deg_res)*deg_res
                                lat_bndhigh = np.ceil(bnd_north/deg_res)*deg_res

                                # Loop through each lat/lon
                                total_pixels = 0
                                bins_latlon_pix = None # bins_latlon_pix shows the number of pixels in each bin for each lat/lon
                                latlon_unique = []
                                for nlon, lon_center in enumerate(np.arange(np.round(lon_bndlow + deg_res/2,2), np.round(lon_bndhigh + deg_res/2,2), deg_res)):
                                    if deg_res == 0.1:
                                        lon_center = np.round(lon_center,2)
                                    lon_idxs = np.where((lon_values >= np.round(lon_center - deg_res/2,2)) & (lon_values < np.round(lon_center + deg_res/2,2)))[0]


                                    if len(lon_idxs) > 0:

                                        for nlat, lat_center in enumerate(np.arange(np.round(lat_bndlow + deg_res/2,2), np.round(lat_bndhigh + deg_res/2,2), deg_res)):
                                            if deg_res == 0.1:
                                                lat_center = np.round(lat_center,2)
                                            lat_idxs = np.where((lat_values >= np.round(lat_center - deg_res/2,2)) & (lat_values < np.round(lat_center + deg_res/2,2)))[0]

                                            if len(lat_idxs) > 0:
                                                latlon_mask = np.zeros(glac_mask.shape)
                                                latlon_mask[lat_idxs[0]:lat_idxs[-1]+1, lon_idxs[0]:lon_idxs[-1]+1] = 1

                                                latlon_mask = latlon_mask * glac_mask

                                                if latlon_mask.sum() > 0:
                                                    latlon_unique.append((lon_center, lat_center))

                                if nfn%1000 == 0:
                                    print(' ', rgiid, '# latlons:', len(latlon_unique))


                                # ===== PROCESS EACH GLACIER FOR THE UNIQUE LAT/LONS =====
                                # If more than one latlon then process; otherwise, can just take mass change
                                if len(latlon_unique) == 1:
                                    lon_center = latlon_unique[0][0]
                                    lat_center = latlon_unique[0][1]
                                    # Load binned annual volume data
                                    bin_volume_annual = ds.bin_volume_annual.values[0,:,:]
                                    # Aggregate volume
                                    latlon_vol_annual = bin_volume_annual.sum(0)[np.newaxis,:]

                                else:
                                    # ----- PROCESS GLACIERS SPANNING MULTIPLE LAT/LONS -----
                                    # Load binned annual volume data
                                    bin_volume_annual = ds.bin_volume_annual.values[0,:,:]

                                    # Elevation bins from model output
                                    bins_elev_init = ds.bin_surface_h_initial.values[0][::-1]
                                    bins_elev = np.zeros(bins_elev_init.shape[0]+1)
                                    bins_elev_dif = bins_elev_init[1:] - bins_elev_init[0:-1]
                                    bins_elev[0] = bins_elev_init[0] - bins_elev_dif[0]
                                    bins_elev[1:-1] = bins_elev_init[0:-1] + bins_elev_dif/2
                                    # maximum based on DEM or model
                                    bins_elev[-1] = np.max([(dem.copy() * glac_mask).max(), bins_elev_init[-1]])
                                    # minimum based on DEM or model
                                    dem_min_array = dem.copy()*glac_mask
                                    dem_min_array[glac_mask==0] = np.nan
                                    bins_elev[0] = np.min([np.nanmin(dem_min_array)-1, bins_elev[0]])

                                    # Test if bins increase monotonically
                                    monotonic_fail_idxs = np.where(bins_elev[:-1] - bins_elev[1:] > 0)[0]
                                    if len(monotonic_fail_idxs) > 0:
                                        # Replace bins without any glacier volume in them with ensuring that they decrease (avoid monotonic error)
                                        offglac_idx = np.where(bin_volume_annual[:,0][::-1]==0)[0]
                                        # Remove uppermost bins which sometimes have no volume
                                        if bin_volume_annual.shape[0]-1 in offglac_idx:
                                            offglac_idx = offglac_idx[0:-1]
                                        # Remove discontinuous portions of the glacier
                                        glac_idx = np.where(bin_volume_annual[:,0][::-1] > 0)[0]
                                        glac_idx = np.arange(glac_idx[0], glac_idx[-1]+1)
                                        offglac_idx = np.array([x for x in offglac_idx if x not in glac_idx])
                                        bins_dif_lowermost = bins_elev[offglac_idx+2][-1] - bins_elev[offglac_idx+1][-1]
                                        # Ensure lowermost bins decrease monotonically
                                        count = 0
                                        while bins_dif_lowermost < 0:
                                            count += 1
                                            offglac_idx = np.append(offglac_idx, offglac_idx[-1]+1)
                                            bins_dif_lowermost = bins_elev[offglac_idx+2+count][-1] - bins_elev[offglac_idx+1+count][-1]
                                        bins_elev[offglac_idx] = bins_elev[offglac_idx[-1]+1] - (offglac_idx+1)[::-1]*bins_dif_lowermost

                                    # ----- Loop over lat/lons -----
                                    total_pixels = 0
                                    total_pixels_latlonmask = 0
                                    bins_latlon_pix = None # bins_latlon_pix shows the number of pixels in each bin for each lat/lon    

                                    for latlon in latlon_unique:
                                        lon_center = latlon[0]
                                        lat_center = latlon[1]

                                        if debug:
                                            print('lon_center:', lon_center, 'lat_center:', lat_center)

                                        if deg_res == 1:
                                            lon_center = np.round(lon_center,1)
                                        lon_idxs = np.where((lon_values >= np.round(lon_center - deg_res/2,2)) & (lon_values < np.round(lon_center + deg_res/2,2)))[0]

                                        if debug:
                                            print('lon_idxs:', len(lon_idxs), lon_idxs)

                                        if len(lon_idxs) > 0:

                                            if deg_res == 0.1:
                                                lat_center = np.round(lat_center,2)
                                            lat_idxs = np.where((lat_values >= np.round(lat_center - deg_res/2,2)) & (lat_values < np.round(lat_center + deg_res/2,2)))[0]


                                            if debug:
                                                print('lat_idxs:', len(lat_idxs), lat_idxs)

                                            if len(lat_idxs) > 0:
                                                latlon_mask = np.zeros(glac_mask.shape)
                                                latlon_mask[lat_idxs[0]:lat_idxs[-1]+1, lon_idxs[0]:lon_idxs[-1]+1] = 1

                                                latlon_mask = latlon_mask * glac_mask

                                                if debug:
                                                    print('latlon_mask sum:', latlon_mask.sum())
                                                total_pixels_latlonmask += latlon_mask.sum()

                                                if latlon_mask.sum() > 0:
                                                    dem_masked = dem.copy()
                                                    dem_masked[latlon_mask==0] = np.min(bins_elev)+1000

                                                    # Flatten array for histogram to work
                                                    dem_masked_flattened = np.reshape(dem_masked, -1)

                                                    # Bin by elevation
                                                    z1_bin_counts, z1_bin_edges = np.histogram(dem_masked_flattened, bins=bins_elev)

                                                    # Record count in each bin for each lat/lon, but flip to get back aligned with ds_binned bins
                                                    if bins_latlon_pix is None:
                                                        bins_latlon_pix = z1_bin_counts[::-1][:,np.newaxis]
                                                    else:
                                                        bins_latlon_pix = np.concatenate((bins_latlon_pix, z1_bin_counts[::-1][:,np.newaxis]), axis=1)

                                                    if debug:
                                                        print('lon/lat:', lon_center, lat_center, 'count:', z1_bin_counts.sum())

                                                    total_pixels += z1_bin_counts.sum()

                                    assert (glac_mask.sum() - total_pixels) / glac_mask.sum() * 100 < 0.1, 'not all pixels counted'

                                    # Calculate the percent area in each unique lat/lon 
                                    bins_latlon_areafrac = np.zeros(bins_latlon_pix.shape)
                                    bins_pix_total = bins_latlon_pix.sum(1)
                                    bins_glac_idx = np.where(bins_pix_total>0)[0]
                                    bins_latlon_areafrac[bins_glac_idx,:] = bins_latlon_pix[bins_glac_idx,:] / bins_pix_total[bins_glac_idx][:,np.newaxis]

                                    # Compute the volume in each lat/lon
                                    nyears = ds.year.values.shape[0]
                                    nbins = bins_elev_init.shape[0]
                                    nlatlon = len(latlon_unique)
                                    bins_latlon_areafrac_annual = np.zeros((nbins, nlatlon, nyears))
                                    bins_latlon_areafrac_annual[:,:,:] = bins_latlon_areafrac[:,:,np.newaxis]

                                    bins_latlon_vol_annual = np.zeros((nbins, nlatlon, nyears))
                                    for nyear in np.arange(0,nyears):
                                        bins_latlon_vol_annual[:,:,nyear] = (bin_volume_annual[:,nyear][:,np.newaxis] 
                                                                             * bins_latlon_areafrac_annual[:,:,nyear])

                                    # Aggregate bins to lat/lon
                                    latlon_vol_annual = bins_latlon_vol_annual.sum(0)

                                    assert (latlon_vol_annual[:,100].sum() - bin_volume_annual[:,100].sum()) < 1, 'Volume change differs'

                                # ===== Export data =====
                                years = ds.year.values
                                latlon_vol_annual_df = pd.DataFrame(latlon_vol_annual, columns=years, index=latlon_unique)
                                latlon_vol_annual_df.to_csv(latlon_csv_fp_rcpgcm + latlon_vol_annual_fn)

                    except:
                        # LOG FAILURE
                        fail_domain_fp = latlon_csv_fp + 'fail/'
                        if not os.path.exists(fail_domain_fp):
                            os.makedirs(fail_domain_fp, exist_ok=True)
                        txt_fn_fail = rgiid + '_' + gcm_name + '_' + rcp + '.csv'
                        with open(fail_domain_fp + txt_fn_fail, "w") as text_file:
                            text_file.write(rgiid + ' failed to complete processing')

            # Delete binned files once processed
            for netcdf_fn in netcdf_fns:
                os.remove(netcdf_fp + netcdf_fn)
                

15.00001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.02001 # latlons: 1
15.03001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.03001 # latlons: 1
15.04001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.04001 # latlons: 1
15.05001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_BCC-CSM2-MR_ssp126_MCMC_ba1_50sets_2000_2100_binned

15.05001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.11001 # latlons: 1
15.12001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.12001 # latlons: 2
15.13001_GFDL-ESM4_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.13001 # latlons: 1
15.00001_INM-CM4-8_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_INM-CM4-8_ssp126_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 #

15.09001_BCC-CSM2-MR_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_BCC-CSM2-MR_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_BCC-CSM2-MR_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.11001 # latlons: 1
15.12001_BCC-CSM2-MR_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.12001 # latlons: 2
15.13001_BCC-CSM2-MR_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.13001 # latlons: 1
15.00001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.02001 # latlons: 1
15.03001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.03001 # latlons: 1
15.04001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.04001 # latlons: 1
15.05001_CESM2_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15

15.00001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.02001 # latlons: 1
15.03001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.03001 # latlons: 1
15.04001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.04001 # latlons: 1
15.05001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_INM-CM4-8_ssp245_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 #

15.05001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.11001 # latlons: 1
15.12001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.12001 # latlons: 2
15.13001_CESM2_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.13001 # latlons: 1
15.00001_CESM2-WACCM_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_CESM2-WACCM_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_CESM2-WACCM

15.10001_INM-CM4-8_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_INM-CM4-8_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.11001 # latlons: 1
15.12001_INM-CM4-8_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.12001 # latlons: 2
15.13001_INM-CM4-8_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.13001 # latlons: 1
15.00001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.02001 # latlons: 1
15.03001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.03001 # latlons: 1
15.04001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.04001 # latlons: 1
15.05001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_INM-CM5-0_ssp370_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 #

15.01001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.01001 # latlons: 1
15.02001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.02001 # latlons: 1
15.03001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.03001 # latlons: 1
15.04001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.04001 # latlons: 1
15.05001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_CESM2-WACCM_ssp585_MCMC_ba1_50sets_2000_2100_binned

15.05001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.05001 # latlons: 1
15.06001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.06001 # latlons: 1
15.07001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.07001 # latlons: 2
15.08001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.08001 # latlons: 1
15.09001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.09001 # latlons: 1
15.10001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.10001 # latlons: 2
15.11001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.11001 # latlons: 1
15.12001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.12001 # latlons: 2
15.13001_INM-CM5-0_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.13001 # latlons: 1
15.00001_MPI-ESM1-2-HR_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15.00001 # latlons: 1
15.01001_MPI-ESM1-2-HR_ssp585_MCMC_ba1_50sets_2000_2100_binned.nc
  RGI60-15

<font face="Calibri" size="3"> <b>Aggregate glaciers latlon annual volumes</b> </font>

In [5]:
for reg in regions:
    for rcp in rcps:
        for gcm_name in gcm_names:
            
            latlon_csv_fp_agg = latlon_csv_fp + '/aggregated/' + str(reg).zfill(2) + '/'
            if not os.path.exists(latlon_csv_fp_agg):
                os.makedirs(latlon_csv_fp_agg)
            latlon_mass_annual_all_fn = str(reg).zfill(2) + '_latlon_mass_kg_annual_' + gcm_name + '_' + rcp + '_' + str(deg_res).replace('.','pt') + 'deg.csv'
            
            # Only process datasets that haven't been processed yet
            if not os.path.exists(latlon_csv_fp_agg + latlon_mass_annual_all_fn):
            
                latlon_csv_fp_rcpgcm = latlon_csv_fp + str(reg).zfill(2) + '/' + gcm_name + '/' + rcp + '/'

                latlon_csv_fns = []
                for i in os.listdir(latlon_csv_fp_rcpgcm):
                    if i.endswith('.csv'):
                        latlon_csv_fns.append(i)

                latlon_csv_fns = sorted(latlon_csv_fns)

                latlon_vol_annual_all = None
                latlon_unique = []
                for nfn, latlon_csv_fn in enumerate(latlon_csv_fns):
                    if nfn%1000 == 0:
                        print(latlon_csv_fn)

                    latlon_vol_annual_df = pd.read_csv(latlon_csv_fp_rcpgcm + latlon_csv_fn, index_col=0)
                    latlon_vol_annual_ind = latlon_vol_annual_df.values
                    latlons = list(latlon_vol_annual_df.index.values)

                    if latlon_vol_annual_all is None:
                        # Record values
                        latlon_vol_annual_all = latlon_vol_annual_df.values
                        latlon_unique = latlons
                    else:
                        # Check if latlon already in the list or not
                        for nidx, latlon in enumerate(latlons):
                            # If not in list, then append latlon to later reference and concatenate array
                            if not latlon in latlon_unique:
                                latlon_unique.append(latlon)
                                latlon_vol_annual_all = np.concatenate((latlon_vol_annual_all, 
                                                                        latlon_vol_annual_ind[nidx,:][np.newaxis,:]), 
                                                                       axis=0)
                            else:
                                latlon_idx = latlon_unique.index(latlon)
                                latlon_vol_annual_all[latlon_idx,:] = latlon_vol_annual_all[latlon_idx,:] + latlon_vol_annual_ind[nidx,:]

    #             print('latlons_unique:', latlon_unique)
                latlon_mass_annual_all = latlon_vol_annual_all * 900
                latlon_mass_annual_df_all = pd.DataFrame(latlon_mass_annual_all, index=latlon_unique, columns=latlon_vol_annual_df.columns)
                latlon_mass_annual_df_all = latlon_mass_annual_df_all.sort_index()
                latlon_mass_annual_df_all.to_csv(latlon_csv_fp_agg + latlon_mass_annual_all_fn)
                        

RGI60-15.00001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.01001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.02001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.03001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.04001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.05001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.06001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.07001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.08001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.09001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.10001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.11001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.12001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.13001_latlon_vol_annual_BCC-CSM2-MR_ssp126.csv
RGI60-15.00001_latlon_vol_annual_CESM2_ssp126.csv
RGI60-15.01001_latlon_vol_annual_CESM2_ssp126.csv
RGI60-15.02001_latlon_vol_annual_CESM2_ssp126.csv
RGI60-15.03001_latlon_vol_annual_CESM2_ssp126.csv
RGI60-15.04001_l

RGI60-15.09001_latlon_vol_annual_MRI-ESM2-0_ssp126.csv
RGI60-15.10001_latlon_vol_annual_MRI-ESM2-0_ssp126.csv
RGI60-15.11001_latlon_vol_annual_MRI-ESM2-0_ssp126.csv
RGI60-15.12001_latlon_vol_annual_MRI-ESM2-0_ssp126.csv
RGI60-15.13001_latlon_vol_annual_MRI-ESM2-0_ssp126.csv
RGI60-15.00001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.01001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.02001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.03001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.04001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.05001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.06001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.07001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.08001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.09001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.10001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.11001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.12001_latlon_vol_annual_NorESM2-MM_ssp126.csv
RGI60-15.1

RGI60-15.05001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.06001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.07001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.08001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.09001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.10001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.11001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.12001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.13001_latlon_vol_annual_MPI-ESM1-2-HR_ssp245.csv
RGI60-15.00001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.01001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.02001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.03001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.04001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.05001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.06001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.07001_latlon_vol_annual_MRI-ESM2-0_ssp245.csv
RGI60-15.08001_latlon_vol_annual_MRI-E

RGI60-15.00001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.01001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.02001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.03001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.04001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.05001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.06001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.07001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.08001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.09001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.10001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.11001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.12001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.13001_latlon_vol_annual_INM-CM5-0_ssp370.csv
RGI60-15.00001_latlon_vol_annual_MPI-ESM1-2-HR_ssp370.csv
RGI60-15.01001_latlon_vol_annual_MPI-ESM1-2-HR_ssp370.csv
RGI60-15.02001_latlon_vol_annual_MPI-ESM1-2-HR_ssp370.csv
RGI60-15.03001_latlon_vol_annual_MPI-ESM1-2-HR_ssp370.csv
RGI60-15.040

RGI60-15.09001_latlon_vol_annual_GFDL-ESM4_ssp585.csv
RGI60-15.10001_latlon_vol_annual_GFDL-ESM4_ssp585.csv
RGI60-15.11001_latlon_vol_annual_GFDL-ESM4_ssp585.csv
RGI60-15.12001_latlon_vol_annual_GFDL-ESM4_ssp585.csv
RGI60-15.13001_latlon_vol_annual_GFDL-ESM4_ssp585.csv
RGI60-15.00001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.01001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.02001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.03001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.04001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.05001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.06001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.07001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.08001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.09001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.10001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.11001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.12001_latlon_vol_annual_INM-CM4-8_ssp585.csv
RGI60-15.13001_latlon_vol_an

In [ ]:
# im = thick_rs.read(1)
# rast_bounds = rasterio.open(oggm_gdirs_fp + thick_fn.replace('.tif', '-reprojected.tif')).bounds # left, bottom, right, top
# fig_bounds = [rast_bounds[0], rast_bounds[2], rast_bounds[1], rast_bounds[3]] # left, right, bottom, top
# title = 'Example Lat/Lon Mask'
# color = 'bone_r'
# ctitle = 'meters'
# show_fig(im, title, color, ctitle, bounds=fig_bounds, res=1)

# im = rasterio.open(oggm_gdirs_fp + dem_fn.replace('.tif', '-reprojected.tif')).read(1)
# rast_bounds = rasterio.open(oggm_gdirs_fp + dem_fn.replace('.tif', '-reprojected.tif')).bounds # left, bottom, right, top
# fig_bounds = [rast_bounds[0], rast_bounds[2], rast_bounds[1], rast_bounds[3]] # left, right, bottom, top
# title = 'Example Thick'
# color = 'bone_r'
# ctitle = 'meters'
# show_fig(im, title, color, ctitle, bounds=fig_bounds, res=1)

In [ ]:
# print('check volume with stats:')
# A = xr.open_dataset('/Users/drounce/Documents/HiMAT/spc_backup/simulations-ind/01/BCC-CSM2-MR/ssp126/stats/1.22193_BCC-CSM2-MR_ssp126_MCMC_ba1_1sets_2000_2100_all.nc')
# for n in np.arange(0,101):
#     print(A.glac_volume_annual.values[0,n] - ds.bin_volume_annual.values[0,:,n].sum(), 'stats volume:', A.glac_volume_annual.values[0,n], 'bin volume:', ds.bin_volume_annual.values[0,:,n].sum())

# another check
# np.abs(latlon_vol_annual.sum(0) - A.glac_volume_annual.values[0,:]).max()

In [ ]:
# # Replace bins that do not increase monotonically
# monotonic_fail_idxs = np.where(bins_elev[:-1] - bins_elev[1:] > 0)[0]
# if len(monotonic_fail_idxs) == 1:
#     for idx in monotonic_fail_idxs:
#         bins_elev[idx] = (bins_elev[idx-1] + bins_elev[idx+1])/2
# elif len(monotonic_fail_idxs) > 1:
#     monotonic_fail_idxs_array = np.array(monotonic_fail_idxs)
#     if all(i > 1 for i in monotonic_fail_idxs[1:] - monotonic_fail_idxs[0:-1]):
#         for idx in monotonic_fail_idxs:
#             bins_elev[idx] = (bins_elev[idx-1] + bins_elev[idx+1])/2